# 🏁 Initialize

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import datetime

PATH = os.path.abspath('')

## Twitter tweepy client

In [2]:
API_KEY = os.getenv('TWITTER_BOT_KEY')
API_SECRET_KEY = os.getenv('TWITTER_BOT_SECRET_KEY')

ACCESS_TOKEN = os.getenv('lree9_ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('lree9_ACCESS_TOKEN_SECRET')

In [3]:
import tweepy

auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
try:
    api.verify_credentials()
except:
    raise Exception("Authentication not OK")

me = api.me()
# me.screen_name

## MongoDB client

In [4]:
MONG_USER = os.getenv('MONG_USER')
MONG_PWD  = os.getenv('MONG_PWD')
MONG_HOST = os.getenv('MONG_HOST')
MONG_PORT = os.getenv('MONG_PORT')
DATABASE = "dataDumpingGround"

In [5]:
from pymongo import MongoClient
from bson.objectid import ObjectId

uri = f"mongodb://{MONG_USER}:{MONG_PWD}@{MONG_HOST}:{MONG_PORT}/{DATABASE}"
client = MongoClient(uri)
db = client[DATABASE]
db

Database(MongoClient(host=['171.244.50.232:27017'], document_class=dict, tz_aware=False, connect=True), 'dataDumpingGround')

# 🎬 Action

In [6]:
import pprint
from helperfunctions import (
    compress_object,
    prettyprint,
    strip_tags
)
from datetime import datetime, timezone, timedelta

## Params

In [16]:
from sys import maxsize as inf
LIMIT = 100 # inf
USER_ID = 'DjMuffinTops'

## Get user info

In [17]:
user = api.get_user(USER_ID)
user_detail = user._json

In [18]:
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/user
attrs_to_save = [
    'id', 'name', 'screen_name', 'location', 'url', 'description',
    'protected', 'verified', 'followers_count', 'friends_count', 'listed_count',
    'favourites_count', 'statuses_count', 'created_at', 'profile_banner_url', 
    'profile_image_url_https', 'default_profile', 'default_profile_image',
    'withheld_in_countries', 'withheld_scope'
]

my_user_obj = compress_object(user._json, attrs_to_save)
my_user_obj['Last updated'] = datetime.now(tz=timezone(timedelta(hours=+7)))
my_user_obj

{'Absent keys': ['withheld_in_countries', 'withheld_scope'],
 'id': 455259919,
 'name': 'DjMuffinTops',
 'screen_name': 'DjMuffinTops',
 'location': 'Boston, MA',
 'url': 'https://t.co/OUpZcI2JOX',
 'description': "Say what you have to say, it's just the way it goes | He/Him | 22 | Digital Artist | Software Engineer | Mystery Dungeon Explorer | ¿music? | Expect many Riolu",
 'protected': False,
 'verified': False,
 'followers_count': 2058,
 'friends_count': 523,
 'listed_count': 15,
 'favourites_count': 12500,
 'statuses_count': 5148,
 'created_at': 'Wed Jan 04 22:34:54 +0000 2012',
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/455259919/1595322921',
 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1342870294608957445/qox5jBI0_normal.jpg',
 'default_profile': False,
 'default_profile_image': False,
 'Last updated': datetime.datetime(2021, 1, 18, 6, 48, 14, 847887, tzinfo=datetime.timezone(datetime.timedelta(seconds=25200)))}

Save to collection "profile"

In [19]:
profile_col = db["twitterProfiles"]
key = {'id': my_user_obj['id']}
profile_col.update(key, my_user_obj, upsert=True)

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

## Get tweets

In [20]:
'''
with open(os.path.join(PATH, f'{USER_ID}_tweet_dump.txt'), 'w+', encoding='utf8') as file:
    for tweet in tweepy.Cursor(api.user_timeline, id=USER_ID).items(LIMIT):
        # parsed = json.loads(tweet._json)
        print(json.dumps(tweet._json, indent=4, sort_keys=True))
        # print(tweet)
        # print(dir(tweet))
        file.write(tweet.text +'\n')
'''
pass

In [21]:
# https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet
attrs = [
    'id', 'text', 'attachments', 'author_id', 'conversation_id', 'created_at',
    'geo', 'in_reply_to_user_id', 'lang', 'possiby_sensitive', 'reply_settings', 
    'source', 'withheld',
    'entities', 'non_public_metrics', 'organic_metrics', 'promoted_metrics', 'public_metrics',
    'referenced_tweets', 'context_annotations', 
]
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet
attrs = [
    'id', 'full_text', 'coordinates', 'place',
    'in_reply_to_status_id', 'in_reply_to_user_id', 'quoted_status_id', 'quoted_status',
    'quote_count', 'reply_count', 'retweet_count', 'favorite_count',
    'entities', 'extended_entities', 'possibly_sensitive', 'filter_level', 'lang', 'matching_rules'
]

In [22]:
from tqdm import tqdm

tweet_col = db["twitterTweets"]

for tweet in tqdm( tweepy.Cursor(api.user_timeline, id=USER_ID, tweet_mode='extended').items(LIMIT) ):
    tweet_obj = compress_object(tweet._json, attrs)
    tweet_obj['Last updated'] = datetime.now(tz=timezone(timedelta(hours=+7)))
    tweet_obj['Author_id'] = tweet._json['user']['id']
    tweet_obj['Source'] = strip_tags(tweet._json['source'])
    tweet_obj['Retweeted_status_id'] = tweet._json['retweeted_status']['id'] if 'retweeted_status' in tweet._json else None

    # prettyprint(tweet._json)
    # prettyprint(tweet_obj)

    key = {'id': tweet_obj['id']}
    tweet_col.update(key, tweet_obj, upsert=True)

100it [00:09, 10.27it/s]
